In [ ]:
# PDTB 句间关系分类 LoRA 微调实验
本实验使用 `MindSpore + mindnlp + LoRA` 在 `DeepSeek-R1-Distill-Qwen-1.5B` 模型上进行微调，任务目标是：
- 输入：一个句子（或对话内容）
- 输出：该句子属于哪一种 PDTB 分类，并解释原因。

In [1]:
import mindnlp
import mindspore
from mindnlp import core
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/python3.10.14/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress 

In [3]:
# 实例化tokenizer
tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', use_fast=False, trust_remote_code=True)
tokenizer

LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151648: AddedToken("<think>", rstrip=False

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
    ms_dtype=mindspore.bfloat16,
    device_map=0
)

model.enable_input_require_grads()  # 开启梯度检查点

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [6]:
# 训练好的 checkpoint 路径
lora_path = "/home/ma-user/work/output/checkpoint-1380"

# 保存合并后模型的目录
merged_path = "/home/ma-user/work/merged_qwen_lora"

In [7]:
# 加载 LoRA 权重
model = PeftModel.from_pretrained(model, lora_path)

In [8]:
# 合并权重
model = model.merge_and_unload()

In [9]:
# 保存最终模型
model.save_pretrained(merged_path)
tokenizer.save_pretrained(merged_path)

print(f"LoRA 权重已合并完成，模型保存在: {merged_path}")

LoRA 权重已合并完成，模型保存在: /home/ma-user/work/merged_qwen_lora


In [11]:
# host to device
model = model.npu()

prompt = "月亮又圆又亮，所以古人称之为玉盘。"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "你是一个中文篇章句间关系分析师"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="ms",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with core.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


嗯，用户给了一个句子，让我分析句间关系。句子是：“月亮又圆又亮，所以古人称之为玉盘。”首先，我需要理解这句话的结构和逻辑关系。句子的结构是并列句式，前后两部分分别描述了月亮的圆亮和古人对其的称呼，然后通过“所以”引出结论，说明古人对月亮的称呼是“玉盘”。

接下来，我要分析句间关系的类型。这里使用了因果关系，因为前半部分描述了月亮的特征，后半部分通过“所以”说明了古人对其的称呼，两者之间存在因果关系。此外，还涉及到因果关系的扩展，因为古人对月亮的称呼也反映了他们对月亮的敬仰和赞美。

然后，我需要考虑句子的语义功能。这句话通过因果关系和扩展语义功能，强调了月亮的圆亮和古人对其的称呼之间的联系，从而增强了文章的连贯性和逻辑性。

最后，总结句间关系的类型是因果关系和扩展语义功能，这有助于更好地理解句子的结构和逻辑关系，从而在写作时能够更好地运用这些关系来增强文章的表达效果。
</think>

句间关系：因果关系和扩展语义功能

原因：前半句话描述了月亮的圆亮特征，后半句话通过“所以”引出古人对其的称呼，形成了因果关系。此外，后半句话还扩展了前半句话的语义功能，强调了月亮的圆亮与古人对其的称呼之间的联系，从而增强了文章的连贯性和逻辑性。
